In [ ]:
# Clone the repo
!git clone https://github.com/CSAILVision/semantic-segmentation-pytorch

In [ ]:
# Mount google drive. Since the data is lost everytime colab is reset, we will store
# the training data and checkpoints in Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
# Installing kaggle api to download data from kaggle
!pip install -q kaggle

# You will need kaggle authentication json file to download data
# See https://github.com/Kaggle/kaggle-api
# "To use the Kaggle API, sign up for a Kaggle account at https://www.kaggle.com. 
# Then go to the 'Account' tab of your user profile (https://www.kaggle.com/<username>/account) and 
# select 'Create API Token'. This will trigger the download of kaggle.json, a file containing your API credentials."

# upload kaggle json file
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# downloading data from kaggle
!kaggle datasets download -d bulentsiyah/semantic-drone-dataset -p /content/gdrive/My\ Drive/drone_sseg_training_data

In [ ]:
# unzipping the downloaded data
!mkdir /content/gdrive/MyDrive/drone_sseg_training_data/drone_data
!unzip -q /content/gdrive/MyDrive/drone_sseg_training_data/semantic-drone-dataset.zip -d /content/gdrive/MyDrive/drone_sseg_training_data/drone_data


In [ ]:
# The RGB images in the training data are JEPGs, but the pipeline expects them in PNG format
# convert all JPGs to PNGs and resize to fit into GPU. 6000*4000 are really big for a GPU
from PIL import Image
import os
from tqdm import tqdm

img_dir = "/content/gdrive/MyDrive/drone_sseg_training_data/drone_data/dataset/semantic_drone_dataset/original_images"
for img in tqdm(os.listdir(img_dir)):
  img_path = os.path.join(img_dir, img)
  im = Image.open(img_path)
  im = im.resize((1200,800), Image.ANTIALIAS)
  im.save(img_path.replace(".jpg", ".png"))
  os.remove(img_path)

In [ ]:
# resizing labels
img_dir = "/content/gdrive/MyDrive/drone_sseg_training_data/drone_data/dataset/semantic_drone_dataset/label_images_semantic"
for img in tqdm(os.listdir(img_dir)):
  img_path = os.path.join(img_dir, img)
  im = Image.open(img_path)
  im = im.resize((1200,800), Image.ANTIALIAS)  
  

In [ ]:
# create .odgt files for training, and split data into training and validation sets
# TODO: add script 

In [ ]:
# installing additional dependencies
!pip install yacs

In [ ]:
# Optional: Connecting colab with ssh. I like to monitor GPU usage using "watch -n1 nvidia-smi"
!pip install colab_ssh --upgrade
from colab_ssh import launch_ssh
launch_ssh("your ngrok id", "some password")
# Run following command in your terminal (replace the port and tcp number)
# ssh -p 15892 root@0.tcp.ngrok.io

In [ ]:
# Time to start training
# A few changes: (a). The pipeline was originally designed to run on multiple GPUs.
# Since we will be running on a single GPU, change the train.py according to this issue - https://github.com/CSAILVision/semantic-segmentation-pytorch/issues/58
# Update the paths and other parameters in the config file. You can also use the one I have uploaded in the config folder
import os
os.chdir('/content/semantic-segmentation-pytorch')
!python train.py --cfg ade20k-hrnetv2.yaml --gpus 0

In [ ]:
# Run model on valdation images
# Note that the test.py expects JPEGs only. 
# You can update the find_recursive function in mit_semseg/utils.py to run on images of your desired format.
!python test.py --cfg config/drone_sseg.yaml --gpu 0 --imgs /content/gdrive/MyDrive/drone_sseg_training_data/drone_data/dataset/semantic_drone_dataset/validation/images